In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical

In [2]:
# DF de entrenamiento, con todas las variables y la columna tier completa
df_train = pd.DataFrame(columns=("data", "tier"))
df_train.loc[0] = [50000, 1]
df_train.loc[1] = [49000, 2]
df_train.loc[2] = [40000, 2]
df_train.loc[3] = [39000, 3]
df_train.loc[4] = [30000, 3]
df_train.loc[5] = [21000, 3]
df_train.loc[6] = [20000, 4]
df_train.loc[7] = [11000, 4]
df_train.loc[8] = [10000, 5]


# DF de test, con todas las variables y la columna tier completa
df_test = pd.DataFrame(columns=("data", "tier"))
df_test.loc[0] = [80000, 1]
df_test.loc[1] = [37000, 3]
df_test.loc[2] = [32000, 3]
df_test.loc[3] = [18000, 4]
df_test.loc[4] = [500, 5]
df_test.loc[5] = [200, 5]
df_test.loc[6] = [45000, 2]

## Preparación de datos

In [3]:
# Separar características y etiquetas
X_train = df_train.drop('tier', axis=1)
y_train = df_train['tier']

X_test = df_test.drop('tier', axis=1)
y_test = df_test['tier']

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convertir etiquetas a categorías
y_train = to_categorical(y_train - 1, num_classes=5)
y_test = to_categorical(y_test - 1, num_classes=5)

## Definición del modelo

In [4]:
from keras.models import Sequential
from keras.layers import Input, Dense

In [5]:
# Definir el modelo
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(5, activation='softmax')
])

# Compilar el modelo
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
)

## Entrenamiento y evaluación del modelo

In [6]:
# Aquí te he hecho un entrenamiento con unos parámetros a ojo, pero es solo 
# una prueba.

# Lo suyo sería que entrenaras con distintos epochs, validation_splits y 
# batch_size para encontrar el "punto dulce" de precisión del modelo.

# Entrenar el modelo
history = model.fit(
    X_train, y_train, epochs=80, validation_split=0.2, batch_size=1
)

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/80
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.1381 - loss: 1.6379 - val_accuracy: 0.0000e+00 - val_loss: 1.6546
Epoch 2/80
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5363 - loss: 1.5716 - val_accuracy: 0.0000e+00 - val_loss: 1.6436
Epoch 3/80
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2717 - loss: 1.5505 - val_accuracy: 0.0000e+00 - val_loss: 1.6472
Epoch 4/80
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6717 - loss: 1.4750 - val_accuracy: 0.0000e+00 - val_loss: 1.6570
Epoch 5/80
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3488 - loss: 1.5085 - val_accuracy: 0.0000e+00 - val_loss: 1.6507
Epoch 6/80
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2717 - loss: 1.4700 - val_accuracy: 0.0000e+00 - val_loss: 1.6508
Epoch 7/80
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4801 - loss: 1.4407 - val_accuracy: 0.0000e+00 - val_loss: 1.6630
Epoch 8/80
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6908 - loss: 1.3568 - val_accuracy: 0.000

## Predicción de tiers

In [7]:
# DF con datos pero sin valor de tiers
df_score = pd.DataFrame(columns=("data",))

df_score.loc[0] = [100000]
df_score.loc[1] = [45500]
df_score.loc[2] = [27900]
df_score.loc[3] = [12000]
df_score.loc[4] = [150]

In [ ]:
# Normalizar los datos de df_score
X_score = df_score.values
X_score = scaler.transform(X_score)

# Predecir los tiers
predictions = model.predict(X_score)
predicted_tiers = predictions.argmax(axis=1) + 1

# Añadir las predicciones a df_score
df_score['predicted_tier'] = predicted_tiers


In [9]:
df_score

,data,predicted_tier
0,100000,1
1,45500,2
2,27900,3
3,12000,4
4,150,4
